In [ ]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.playlistItems.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from IPython.display import clear_output

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

In [ ]:
# Setup
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name    = "youtube"
api_version         = "v3"
client_secrets_file = "../keys/client_secrets.json"

def get_flow_credentials_yt( client_secrets_file, scopes ):
    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, 
        scopes
    )
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, 
        api_version, 
        credentials = credentials
    )
    clear_output( wait = False ) # Prevent keys from being stored in output
    return flow, credentials, youtube

flow, credentials, youtube = get_flow_credentials_yt( client_secrets_file, scopes )

In [ ]:
from pprint import pprint

def fetch_entire_playlist( playlistId, part ):
    """ Send serial requests until info about all playlist items is obtained """
    
    # Init
    request = youtube.playlistItems().list(
        playlistId = playlistId ,
        part       = part ,
        maxResults = 50
    )
    response = request.execute()
    nextPage = response["nextPageToken"]
    items    = response['items']
    last     = False
    
    while 1:
        request = youtube.playlistItems().list(
            playlistId = "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i",
            part       = "contentDetails,id,snippet,status",
            pageToken  = nextPage,
            maxResults = 50
        )
        response = request.execute()
        try:
            nextPage = response["nextPageToken"]
        except KeyError:
            last = True
        items.extend( response['items'] )
        if last:
            break
        
    return items
    

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    
    if 0:
        request = youtube.playlistItems().list(
            playlistId = "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i",
            part       = "contentDetails,id,snippet,status",
            maxResults = 50
        )
        response = request.execute()
        for res in response:
            print( res )
        print( f"Next Page: {response['nextPageToken']}" )
        items = response['items']
    else:
        playlistId = "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i" 
        part       = "contentDetails,id,snippet,status" 
        items = fetch_entire_playlist( playlistId, part )
    
    N      = len( items )
    public = 0
    
    
    
    for res in items:
        if res['status']['privacyStatus'] == 'public':
            public += 1
        else:
            print( f"Nonpublic Video: {res['status']['privacyStatus']}" )
            
    print( f"{public} / {N} items are public!" )

    print( '\n\n==== ALL RESULTS ====\n' )
    for res in items:
        pprint( res )
        print( '\n~~~\n' )

In [ ]:
if __name__ == "__main__":
    main()